In [90]:
import os
from glob import glob
import wfdb
import numpy as np
import pandas as pd
import scipy
from tqdm import tqdm

In [91]:
root = "/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/data/PTB_XL"

In [92]:
# hr_records = []

# original_data_source = "/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/Datasets/PTB_XL_data"
# with open(original_data_source + "/RECORDS" , 'r') as f:
#     records = f.readlines()
# for r in records:   
#     if "records500" in r:
#         hr_records.append(r.strip("\n"))

# for r in hr_records:
#     data , m = wfdb.rdsamp(original_data_source + "/" + r)
#     dict = {}
#     dict['val'] = data
#     file_name = r.split("/")[-1].split("_")[0] + ".mat"
#     scipy.io.savemat(file_name = file_name , mdict = dict)

In [93]:
data_records = []

with open(os.path.join(root, "RECORDS"), "r") as f:
    for line in f:
        if "records500" in line:
            data_records.append(os.path.join(root , line.strip()))

In [94]:
meta_info = pd.read_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/data/PTB_XL/ptbxl_database.csv")
meta_info['scp_codes']


0                 {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
1                             {'NORM': 80.0, 'SBRAD': 0.0}
2                               {'NORM': 100.0, 'SR': 0.0}
3                               {'NORM': 100.0, 'SR': 0.0}
4                               {'NORM': 100.0, 'SR': 0.0}
                               ...                        
21794    {'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...
21795             {'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
21796                           {'ISCAS': 50.0, 'SR': 0.0}
21797                           {'NORM': 100.0, 'SR': 0.0}
21798                           {'NORM': 100.0, 'SR': 0.0}
Name: scp_codes, Length: 21799, dtype: object

In [95]:
scp_codes_df = pd.read_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_project/data/PTB_XL/scp_statements.csv")
scp_codes_df.rename(columns = {'Unnamed: 0' : 'Potential_diagnosis'} , inplace = True)

In [96]:
print(meta_info.columns)
new_df = meta_info[['filename_hr' , 'ecg_id' , 'age' , 'sex' , 'scp_codes']]

Index(['ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight', 'nurse',
       'site', 'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')


In [97]:
# finding the non-zero values in scp_codes_column
collection_non_zero_strings = []
for row in new_df.iterrows():
    non_zero_string = ""
    dict_string = row[1]['scp_codes']
    dict_string_split = dict_string.split(",")
    dict_string_split = [x.split(":") for x in dict_string_split]
    for x in dict_string_split:
        x[0] = x[0].strip(' ')
        x[0] = x[0].strip("{")
        x[0] = x[0].strip(' ')
        x[0] = x[0].strip('\'')
        x[1] = x[1].strip(' ')
        x[1] = x[1].strip('}')
        x[1] = float(x[1])
        if x[1]!=0:
            non_zero_string = non_zero_string + x[0] + " "
    collection_non_zero_strings.append(non_zero_string)

new_df['potential_diagnosis'] = collection_non_zero_strings
new_df.drop(columns = ['scp_codes'] , inplace = True)
split_diagnosis = new_df['potential_diagnosis'].str.split(" " , expand = True)
new_df = pd.concat((new_df , split_diagnosis) , axis = 1)
new_df.drop(columns = ['potential_diagnosis'] , inplace = True)
new_df.rename(columns = {0 : "diagnosis_0" , 1 : "diagnosis_1" , 2 : "diagnosis_2" , 3 : "diagnosis_3" , 4 : "diagnosis_4" , 5 : "diagnosis_5"} , inplace = True)

/var/folders/x0/r7576wrd5w3dfxhxvxn3v81w0000gn/T/ipykernel_44307/2044714244.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['potential_diagnosis'] = collection_non_zero_strings
/var/folders/x0/r7576wrd5w3dfxhxvxn3v81w0000gn/T/ipykernel_44307/2044714244.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(columns = ['scp_codes'] , inplace = True)


In [98]:
new_df_other_columns = [c for c in new_df.columns if c != "filename_hr"]
new_df['patient_id'] = new_df['filename_hr'].apply(lambda x: x.split("/")[-1].split("_")[0])
new_df_new_columns = ['patient_id'] + new_df_other_columns
new_df = new_df[new_df_new_columns]
new_df.to_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/data/PTB_XL/reference.csv" , index = False , header = True)

In [99]:
# creating a table for the diagnosis vs number of values
columns = ['diagnosis_' + str(i) for i in range(6)]
unique_diagnosis = new_df['diagnosis_0'].unique().tolist()
for col in columns[1:]:
    unique_diagnosis += new_df[col].unique().tolist()
unique_diagnosis = list(set(unique_diagnosis))
unique_diagnosis.remove('')
dict_counts = {}
for diagnosis in unique_diagnosis:
    dict_counts[diagnosis] = []
    for col in columns:
        try:
            dict_counts[diagnosis].append(new_df[col].value_counts()[diagnosis])
        except:
            dict_counts[diagnosis].append(0)

count_df = pd.DataFrame(dict_counts).T
count_df.rename(columns = {i : "diagnosis_" + str(i) for i in range(6)} , inplace = True)
for col in count_df.columns:
    count_df.loc[None , col] =  len(new_df) - count_df[col].sum()
count_df

,diagnosis_0,diagnosis_1,diagnosis_2,diagnosis_3,diagnosis_4,diagnosis_5
INJIN,2,4,12,0,0,0
NORM,9514,0,0,0,0,0
LVH,1212,738,160,20,2,0
INJAS,112,87,14,0,1,0
WPW,73,5,0,1,0,0
ISCIL,78,87,12,2,0,0
ISC_,180,583,402,97,10,0
ANEUR,0,44,41,14,5,0
ILBBB,29,36,10,1,1,0
ISCAS,92,66,8,2,1,0


In [100]:
for col in count_df.columns:
    print(col , count_df[col].sum())

diagnosis_0 21799
diagnosis_1 21799
diagnosis_2 21799
diagnosis_3 21799
diagnosis_4 21799
diagnosis_5 21799


In [101]:
df = pd.read_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/data/CPSC/labels.csv")

In [102]:
df.columns

Index(['patient_id', 'SNR', 'AF', 'IAVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD',
       'STE', 'fold'],
      dtype='object')

In [103]:
label_csv_path ="/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/data/PTB_XL/labels.csv"

In [104]:
new_df

,patient_id,ecg_id,age,sex,diagnosis_0,diagnosis_1,diagnosis_2,diagnosis_3,diagnosis_4,diagnosis_5
0,00001,1,56.0,1,NORM,,None,None,None,None
1,00002,2,19.0,0,NORM,,None,None,None,None
2,00003,3,37.0,1,NORM,,None,None,None,None
3,00004,4,24.0,0,NORM,,None,None,None,None
4,00005,5,19.0,1,NORM,,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
21794,21833,21833,67.0,1,NDT,PVC,,None,None,None
21795,21834,21834,300.0,0,NORM,,None,None,None,None
21796,21835,21835,59.0,1,ISCAS,,None,None,None,None
21797,21836,21836,64.0,1,NORM,,None,None,None,None


In [105]:
# collecting all potential labels
label_csv = new_df['patient_id'].to_frame()

potential_labels = []
for cols in ['diagnosis_'+ str(i) for i in range(6)]:
    potential_labels += new_df[cols].tolist()
    potential_labels = list(set(potential_labels))
potential_labels.remove(None)
potential_labels.remove('')


for label in (potential_labels):
    label_csv[label] = 0
    for col in ['diagnosis_'+ str(i) for i in range(6)]:
        all_with_label1 = new_df[new_df[col] == label]['patient_id'].tolist()      
        label_csv.loc[label_csv['patient_id'].isin(all_with_label1) ,label] = 1

In [106]:
label_csv

,patient_id,NORM,LVH,INJAS,WPW,ISCIL,ISC_,ANEUR,ILBBB,ISCAS,...,INJIL,LPFB,ALMI,PAC,CLBBB,BIGU,INJAL,AFLT,INJIN,AFIB
0,00001,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00002,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,00003,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,00004,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00005,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21794,21833,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21795,21834,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21796,21835,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
21797,21836,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
label_csv[potential_labels].sum()

NORM       9514
LVH        2132
INJAS       214
WPW          79
ISCIL       179
ISC_       1272
ANEUR       104
ILBBB        77
ISCAS       169
ISCLA       140
PVC        1027
IRBBB      1118
ISCAL       659
SEHYP        29
ISCIN       218
PACE        287
3AVB         16
ILMI        478
LMI         201
CRBBB       541
AMI         353
2AVB         14
ASMI       2357
1AVB        793
PMI          17
NDT        1825
IVCD        787
INJLA        17
IPLMI        51
IPMI         33
RVH         126
IMI        2676
NST_        559
EL           96
LAFB       1623
RAO/RAE      99
LNGQT       116
PSVT         24
ISCAN        44
STACH         4
LAO/LAE     426
DIG         177
INJIL        15
LPFB        177
ALMI        288
PAC          37
CLBBB       536
BIGU         24
INJAL       145
AFLT         56
INJIN        18
AFIB         48
dtype: int64

In [108]:
label_csv.shape

(21799, 53)

In [109]:
potential_labels

['NORM',
 'LVH',
 'INJAS',
 'WPW',
 'ISCIL',
 'ISC_',
 'ANEUR',
 'ILBBB',
 'ISCAS',
 'ISCLA',
 'PVC',
 'IRBBB',
 'ISCAL',
 'SEHYP',
 'ISCIN',
 'PACE',
 '3AVB',
 'ILMI',
 'LMI',
 'CRBBB',
 'AMI',
 '2AVB',
 'ASMI',
 '1AVB',
 'PMI',
 'NDT',
 'IVCD',
 'INJLA',
 'IPLMI',
 'IPMI',
 'RVH',
 'IMI',
 'NST_',
 'EL',
 'LAFB',
 'RAO/RAE',
 'LNGQT',
 'PSVT',
 'ISCAN',
 'STACH',
 'LAO/LAE',
 'DIG',
 'INJIL',
 'LPFB',
 'ALMI',
 'PAC',
 'CLBBB',
 'BIGU',
 'INJAL',
 'AFLT',
 'INJIN',
 'AFIB']

In [110]:
label_csv.to_csv(label_csv_path , index = False , header = True)